In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'shd_soa_baselines'

data = test_loader, train_loader

dropout=0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

D:\RESEARCHERS\Alberto\Anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]


## Baseline (no dropout) -> 87.5

In [8]:
train(snn, data, 1e-4, 50, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, scheduler=(1, 0.98))

RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49123
Step [20/31], Loss: 0.47768
Step [30/31], Loss: 0.45534
Time elasped: 39.090970516204834
Epoch [2/50]
Step [10/31], Loss: 0.42956
Step [20/31], Loss: 0.41055
Step [30/31], Loss: 0.39110
Time elasped: 39.346556663513184
Epoch [3/50]
Step [10/31], Loss: 0.37323
Step [20/31], Loss: 0.35470
Step [30/31], Loss: 0.33961
Time elasped: 39.052327156066895
Epoch [4/50]
Step [10/31], Loss: 0.32846
Step [20/31], Loss: 0.31343
Step [30/31], Loss: 0.30237
Time elasped: 38.97873544692993
Epoch [5/50]
Step [10/31], Loss: 0.29187
Step [20/31], Loss: 0.28027
Step [30/31], Loss: 0.27503
Time elasped: 39.00328063964844
Test Loss: 0.031483145164

## Baseline (dropout = 0.2) -> 87.1

In [7]:
hidden = (64, 2, 'f')
depth, stride = 150, 3
surr = 'step'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='sum', device=device)
snn.modelname = modelname

train(snn, data, 1e-4, 50, dropout=0.2, lr_scale=(1.0, 1.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(1, 0.98))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): MSELoss()
  (f0_i): Linear(in_features=700, out_features=64, bias=False)
  (f1_f2): Linear(in_features=3200, out_features=64, bias=False)
  (f2_o): Linear(in_features=3200, out_features=20, bias=False)
)
Total params: 313600
Total mult-adds (M): 78.4
training shd250_64x2_d_150_3_step for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 0.49450
Step [20/31], Loss: 0.47842
Step [30/31], Loss: 0.46485
Time elasped: 56.74550414085388
Epoch [2/50]
Step [10/31], Loss: 0.44598
Step [20/31], Loss: 0.42368
Step [30/31], Loss: 0.40241
Time elasped: 56.640833616256714
Epoch [3/50]
Step [10/31], Loss: 0.38519
Step [20/31], Loss: 0.36767
Step [30/31], Loss: 0.35211
Time elasped: 56.34934186935425
Epoch [4/50]
Step [10/31], Loss: 0.34113
Step [20/31], Loss: 0.3

## fs-mot-48 (150,3):

In [2]:
hidden = (48, 2, 'f')
depth, stride = 150, 3
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=2400, out_features=48, bias=False)
  (f2_o): Linear(in_features=2400, out_features=20, bias=False)
)
Total params: 196800
Total mult-adds (M): 49.2
training shd250_48x2_d_150_3_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.97287
Step [20/31], Loss: 22.66333
Step [30/31], Loss: 16.76681
Time elasped: 59.207849979400635
Epoch [2/50]
Step [10/31], Loss: 12.47277
Step [20/31], Loss: 10.09367
Step [30/31], Loss: 7.37827
Time elasped: 57.588488817214966
Epoch [3/50]
Step [10/31], Loss: 5.94118
Step [20/31], Loss: 5.47659
Step [30/31], Loss: 5.53753
Time elasped: 56.63181257247925
Epoch [4/50]
Step [10/31], Loss: 4.14032
Step [20/3

## fs-mot-48 (150,6):

In [3]:
hidden = (48, 2, 'f')
depth, stride = 150, 6
surr = 'fs'

modelname = f'{dataset}{time_window}_{hidden[0]}x2_d_{depth}_{stride}_{surr}'

snn = RSNN_d_d(dataset, hidden=hidden, delay = (depth, stride), batch_size=batch_size, win=time_window, surr=surr, loss_fn='mot', device=device)
snn.modelname = modelname

train(snn, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))

delays: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144]
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=1200, out_features=48, bias=False)
  (f2_o): Linear(in_features=1200, out_features=20, bias=False)
)
Total params: 115200
Total mult-adds (M): 28.8
training shd250_48x2_d_150_6_fs for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 27.95343
Step [20/31], Loss: 20.03972
Step [30/31], Loss: 13.16850
Time elasped: 55.28170108795166
Epoch [2/50]
Step [10/31], Loss: 8.07913
Step [20/31], Loss: 6.37601
Step [30/31], Loss: 5.47225
Time elasped: 55.048312187194824
Epoch [3/50]
Step [10/31], Loss: 4.50665
Step [20/31], Loss: 4.73025
Step [30/31], Loss: 3.28228
Time elasped: 55.04167151451111
Epoch [4/50]
Step [10/31], Loss: 2.61390
Step [20/31], Loss: 2.28400
Step [30/31], Loss: 2.55172
Time elasped: 55.28854012489319
Epoch [5/50]
Step [10/31], Loss